In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
from nbhd.data import Base

Initializing database connection...
Database connected!


In [4]:
db = Base()

In [5]:
import inspect

In [6]:
src = inspect.getsource(db.nearest_neighbours)

In [7]:
print(src)

    def nearest_neighbours(self, table, boundary_wkt):
        sql = f'''
        SELECT uprn."UPRN",
                uprn.geometry as uprn_geometry,
                dbtable.*,
               ST_Distance(dbtable.geometry, uprn.geometry) AS dist
        FROM ({self._contains_query('openuprn', boundary_wkt)}) AS uprn
        CROSS JOIN LATERAL (
          SELECT dbtable.* 
          FROM ({self._contains_query(table, boundary_wkt)}) AS dbtable 
          ORDER BY dbtable.geometry <-> uprn.geometry
          LIMIT 1
        ) dbtable;
        '''
        return self.query(sql, spatial=True)



In [8]:
def nearest_same(self, table, boundary_wkt, within):
    wkt = self._contains_query(table, boundary_wkt)
    sql = f'''
    SELECT dbtable.id as id_1,
            table1.id as id_2,
    ST_Distance(dbtable.geometry, table1.geometry) AS dist
    FROM ({wkt}) AS table1
    CROSS JOIN LATERAL (
      SELECT id, geometry
      FROM ({wkt}) AS dbtable 
      WHERE ST_Distance(dbtable.geometry, table1.geometry) < {within} 
    ) AS dbtable
    WHERE dbtable.id != table1.id
    ;
    
    '''
    return self.query(sql, spatial=False)

In [9]:
db.ls()

['spatial_ref_sys',
 'slimroads',
 'opennames',
 'openroads',
 'openmaplocal',
 'openuprn',
 'railways',
 'rivers',
 'coastline',
 'roadnodes',
 'bigtilesa']

In [16]:
import pandas as pd

In [10]:
from nbhd.geography import Neighbourhood

In [11]:
n = Neighbourhood()

Getting data took 0 minutes, 0 seconds.


In [12]:
nearest_nodes = nearest_same(db, 'roadnodes', n.geom.buffer(1), 10)

In [14]:
df = nearest_nodes[['id_1','id_2', 'dist']]

In [15]:
df['pair'] = df.apply(lambda x: str(sorted([x.id_1, x.id_2])), axis=1)

In [17]:
translate = {eval(p)[0]: eval(p)[1] for p in df.pair.unique()}

In [18]:
nodes = db.contains('roadnodes', n.geom.buffer(1))

In [19]:
nodes['snapped'] = nodes.id.apply(lambda x: translate.get(x, x))